<a href="https://colab.research.google.com/github/windianaresta/projectakhir/blob/main/Testing_Sentimen_Analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Sastrawi
!pip install nltk
import pickle
from pickle import UnpicklingError
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import requests
import json
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
try:
    with open('/content/svm_model.pkl', 'rb') as f:
        svm = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading xgb model: {e}")

    print("Check if 'svm_model.pkl' exists and is not empty.")
    print("If the file was transferred, ensure it was done in binary mode.")


try:
    with open('/content/tfidf_vectorizer.pkl', 'rb') as f:
        tfidf = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading TF-IDF vectorizer: {e}")


def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join([word for word in text.split() if word.lower() not in ["shopee", "qris", "mobile", "shopeepay"]])
    text = text.strip(' ')
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ya','loh','kah','deh'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

def fix_slangwords(text):
    words = text.split()
    fixed_words = []
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
    fixed_text = ' '.join(fixed_words)
    return fixed_text

url = 'https://raw.githubusercontent.com/windianaresta/projectakhir/main/slangwords.json'  # URL tempat kamus slangwords disimpan

response = requests.get(url)

if response.status_code == 200:
    try:
        slangwords = json.loads(response.text)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        print("Response content:", response.text)
else:
    print("Failed to fetch data from URL. Status code:", response.status_code)

def preprocess_text(text):
    text = cleaningText(text)
    text = casefoldingText(text)
    text = fix_slangwords(text)
    text = tokenizingText(text)
    text = filteringText(text)
    text = toSentence(text)
    return text

def prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm):

    review_baru_cleaned = cleaningText(review_baru)
    review_baru_casefolded = casefoldingText(review_baru_cleaned)
    review_baru_slangfixed = fix_slangwords(review_baru_casefolded)
    review_baru_tokenized = tokenizingText(review_baru_slangfixed)
    review_baru_filtered = filteringText(review_baru_tokenized)
    review_baru_final = toSentence(review_baru_filtered)


    X_review_baru = tfidf.transform([review_baru_final])


    X_review_baru = X_review_baru.toarray()


    prediksi_sentimen = svm.predict(X_review_baru)


    if prediksi_sentimen[0] == 'positive':
        hasil = "Sentimen review baru adalah POSITIF."
    elif prediksi_sentimen[0] == 'negative':
        hasil = "Sentimen review baru adalah NEGATIF."
    else:
        hasil = "Sentimen review baru adalah NETRAL."

    return hasil

In [ ]:
review_baru = "aneh"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah NETRAL.'

In [ ]:
review_baru = "sangat puas"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah POSITIF.'

In [ ]:
review_baru = "jelek"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah NEGATIF.'